In [1]:
from osgeo import gdal

In [2]:
import os
from utils.GenUtils import make_folder, get_paths, chunk_creator, folder_file_size, question

In [3]:
os.environ["ISISROOT"]="/opt/conda/envs/isis/"
os.environ["ISISDATA"]="/isis/data"

In [4]:
import kalasiris as isis

In [5]:
config = {
            "inst":"m3",
            "maptemplate":"./maptemplate/map_template_moon_eqc.map"
            }

In [6]:
!ls ./maptemplate

map_template_moon_eqc.map


In [7]:
if config['inst']=='lronac':
    from utils.KalaUtils import lronac
    proc_func=lronac
    ixt='IMG'
elif config['inst']=='lrowac':
    from utils.KalaUtils import lrowac
    proc_func=lrowac
    ixt='IMG'
elif config['inst']=='m3':
    from utils.KalaUtils import m3
    proc_func=m3
    ixt='LBL'
else:
    print('Cannot find utils')
    
maptemplate=config['maptemplate']

In [8]:
def cub2map(src, proc_func, maptemplate, ixt):
    src_basename = os.path.basename(src).split('.'+ixt)[0]
    dst_basename = DSTPATH+'/'+src_basename
    print(dst_basename)
    if os.path.isfile(dst_basename+'_lev2.JP2'):
        print ("File exist")
    elif os.path.isfile(dst_basename+'.JP2'):
        print ("File exist")
    else:
        proc_func(src, dst_basename, maptemplate)
  

In [9]:
def parallel_cub2map(files, JOBS, proc_func, maptemplate, ixt):
    from joblib import Parallel, delayed
    Parallel (n_jobs=JOBS)(delayed(cub2map)(files[i],proc_func, maptemplate, ixt)
                            for i in range(len(files)))

In [10]:
def main(PATH, DPATH, ixt, maptemplate):
    image_list = get_paths(PATH, ixt) 
    total_size, max_size, av_fsize = folder_file_size(PATH,image_list)
    from tqdm import tqdm
    import psutil
    avram=psutil.virtual_memory().total >> 30
    avcores=psutil.cpu_count(logical=False)
    avthreads=psutil.cpu_count(logical=True)
    ram_thread = avram/avthreads
    req_mem = avthreads*max_size
    if req_mem > avcores and req_mem > avram:
        JOBS = avcores
    else:
        JOBS = avcores
    with tqdm(total=len(image_list),
             desc = 'Generating Images',
             unit='File') as pbar:
        filerange = len(image_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(image_list, JOBS):
            chunks.append(c)
        # from time import time
        from datetime import datetime
        for i in range(len(chunks)):
            start = datetime.now()
            dt_string = start.strftime("%d/%m/%Y %H:%M:%S")
            print(f'Loop {i} started at: {dt_string}', chunks[i])
            files = chunks[i]
            parallel_cub2map(files, JOBS, proc_func, maptemplate, ixt)
            pbar.update(JOBS)

In [ ]:
if __name__ == "__main__":
    PATH = '../data'
    DSTPATH = PATH+'/processed'
    os.makedirs(DSTPATH, exist_ok=True)
    print(ixt)
    main(PATH, DSTPATH, ixt, maptemplate)

LBL


Generating Images:   0%|                                                                                                                                                                   | 0/1 [00:00<?, ?File/s]

Loop 0 started at: 21/01/2022 00:05:06 ('../data/M3G20090107T011405_V02_L1B.LBL',)
../data/processed/M3G20090107T011405_V02_L1B
Creating cube
Map projecting
Projecting cube to map
Mapping
